In [ ]:
#| default_exp rate_limiter

# Rate Limiter

In [ ]:
#| export

from collections import deque, defaultdict
from abc import ABC, abstractmethod
import time

In [ ]:
#| export
#| echo: true

class RateLimiter(ABC):

    def __init__(self, time_window: int=float('inf'), num_requests: int=float('inf')):
        self.time_window = time_window
        self.num_requests = num_requests

    def grant_access(self):
        return "ACCESS_GRANTED"

    def restrict_access(self):
        return "TOO_MANY_REQUESTS"
    
    def is_valid_request(self, user_id):
        if self.is_valid(user_id): return self.grant_access()
        else: return self.restrict_access()
    
    @abstractmethod
    def is_valid(self, user_id):
        pass

    @property
    def current_time(self):
        return time.time()


In [ ]:
#| export
#| echo: true

class SlidingWindowRateLimiter(RateLimiter):
    def __init__(self, time_window: int = float('inf'), num_requests: int = float('inf')):
        super().__init__(time_window, num_requests)
        self.user_id_map = defaultdict(deque)
    
    def is_valid(self, user_id):
        q = self.user_id_map[user_id]
        while q and q[0]<self.current_time-self.time_window:
            q.popleft()
        if len(q)+1<=self.num_requests:
            q.append(self.current_time)
            return True
        return False
    
    def __str__(self):
        return str(self.user_id_map)

In [ ]:
#| export
#| echo: true

class TokenBucketRateLimiter(RateLimiter):
    def __init__(self, time_window: int = float('inf'), num_requests: int = float('inf')):
        super().__init__(time_window, num_requests)
        self.user_id_map = defaultdict(lambda: num_requests)
        self.last_update_time = self.current_time

    def is_valid(self, user_id):
        self.user_id_map[user_id] += int(self.num_requests*(self.current_time-self.last_update_time)/self.time_window)
        self.last_update_time = self.current_time
        if self.user_id_map[user_id]>0:
            self.user_id_map[user_id]-=1
            return True
        return False
    
    def __str__(self):
        return str(self.user_id_map)